In [6]:
from system import *

s1 = system('2',MMA=True)

Initializing system 2  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  2  initialized in 3.0 seconds.


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [7]:
diag = 2  #input n corresponding to diag element

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')


#(gamma_f=gamma)#gamma_g=0,

#Take only one of the compl conj terms. Switch maybe second 0 to 1
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage

    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)

    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)

    var('r_c',domain='real',latex_name =r'r_c')
    b = b.subs(kappa_c=gamma/r_c)


    b = b.subs(g = sqrt(C)*gamma) 

    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full=True)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b

show( symround(h_eff))

(C*r_c + 0.50*I*Det)/((Det*R_f + DEt)*C*r_c + 0.50*I*DEt*Det)

# Expand linearly in the limit:   $C\rightarrow \infty$

In [8]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)
#t = q._mathematica_().Series((x,0,2))
#t_exp = t[3]
#t_st = t[4]
#tayl = 0
#for i in range(1,len(t_exp)):
#    tayl += MMA_simplify( t_exp[i]._sage_() ) / C**(i-1+Integer(t[4]))

show(symround(tayl))

#with sage
#tayl = b.taylor(C_c,infinity, 1)
#show(symround(MMA_simplify(tayl)))

1/(Det*R_f + DEt) + 0.50*I*Det^2*R_f/((Det*R_f + DEt)^2*C*r_c)

# Lindblau Operators

Choose which of the Lindblau operators to view and which non zero element of it

In [10]:
def lind(lind_op,which):    
    L_matrix_orig = s1.lindblau_list[lind_op]
    L_matrix      = s1.eff_lindblau_list[lind_op] #lindblau_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)

    #COUPLINGS
    var('R_f',domain='real')#ratio  (g_f/g)^2
    L_elem = L_elem.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    L_elem = L_elem.subs(v=g*sqrt(R_v))

    #ABSORBTIONS
    var('r_b',domain='real')
    L_elem = L_elem.subs(kappa_b=gamma/r_b)

    var('r_c',domain='real',latex_name =r'r_c')
    L_elem = L_elem.subs(kappa_c=gamma/r_c)

    var('gamma_g')
    var('r_g',domain='real',latex_name =r'r_g')
    L_elem = L_elem.subs(gamma_g=gamma/r_g)

    var('gamma_f')
    var('r_f',domain='real',latex_name =r'r_f')
    L_elem = L_elem.subs(gamma_f=gamma/r_f)

    L_elem = L_elem.subs(g = sqrt(C)*gamma) 

    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))



    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])

lind_op = 1  #which lindblau operator
which   = 1  #which non zero element to access
lind(lind_op,which)

(C*r_c + 0.50*I*Det)*Omega*sqrt(gamma/r_g)/(((Det*R_f + DEt)*C*r_c + 0.50*I*DEt*Det)*gamma)

 |0f01> < 0g01 | 

 Original coefficient :


sqrt(gamma_g)

# Expand linearly in the limit:   $C\rightarrow \infty$

In [5]:
#with mathematica

q = L_elem.subs(C=1/x)
q = MMA_simplify(q)
t = q._mathematica_().Series((x,0,1)).Normal()

tayl = t._sage_().subs(x=1/C)
tayl = SR(str(tayl).replace('Sqrt','sqrt'))
show(symround(tayl))

NameError: name 'L_elem' is not defined

((8.00000000000000*Det*R_f + 16.0000000000000*DEt)*C^2*r_c^2 + 1.00000000000000*DEt*Det^2)/((4.00000000000000*Det^2*R_f^2 + 16.0000000000000*DEt*Det*R_f + 16.0000000000000*DEt^2)*C^2*r_c^2 + 1.00000000000000*DEt^2*Det^2)